In [24]:
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
from pathlib import PurePath, Path
import pyarrow
import scipy.stats as stats
import scipy
from statsmodels.tsa.stattools import adfuller
import time
import sys
import datetime


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC


from book_functions import *

importlib.reload(sys.modules['book_functions'])
# importlib.reload(sys.modules['sklearn'])

<module 'book_functions' from 'C:\\Users\\mlevy\\Workspace\\Advances-in-Financial-Machine-Learning\\F-ML\\book_functions.py'>

In [46]:
from random import gauss
from itertools import product
#———————————————————————————————————————
# 13.1
def main():
    # mesh of profit taking and stop loss pairs
    #rPT = rSLm = np.linspace(0, 10, 21)
    rPT = [5]
    rSLm = [10]
    count = 0
    # Cartesian product of forecasted price and half-life (step 1)
    for prod_ in product([10, 5, 0, -5, -10], [5, 10, 25, 50, 100]):
        print('Forecast: {},  H-L: {}'.format(prod_[0], prod_[1]))
        count += 1
        coeffs = {'forecast': prod_[0], 'hl': prod_[1], 'sigma': 1}
        output = batch(coeffs, nIter=1e5, maxHP=100, rPT=rPT, rSLm=rSLm)
    return output


# 13.2
def batch(coeffs, nIter=1e5, maxHP=100, rPT=np.linspace(.5, 10, 20),
          rSLm=np.linspace(.5, 10, 20), seed=0):
    phi, output1 = 2**(-1./coeffs['hl']), []
    for comb_ in product(rPT, rSLm):  # step 2: mesh of profit-taking and stop-loss pairs
        output2 = []
        for iter_ in range(int(nIter)):
            p, hp, count = seed, 0, 0
            while True:
                p = (1-phi)*coeffs['forecast']+phi * \
                    p+coeffs['sigma']*gauss(0, 1)
                cP = p-seed
                hp += 1
                # step 4
                if cP > comb_[0] or cP < -comb_[1] or hp > maxHP:
                    output2.append(cP)
                    break
        mean, std = np.mean(output2), np.std(output2)
        print('PT: {}, SL: {}, mean: {}, std: {}, SR: {}'.format(comb_[0], comb_[1], mean, std, mean/std))
        output1.append((comb_[0], comb_[1], mean, std, mean/std))
    return output1

In [47]:
main()

Forecast: 10,  H-L: 5
PT: 5, SL: 10, mean: 5.7851606736597185, std: 0.6261185437509011, SR: 9.239721026313067
Forecast: 10,  H-L: 10
PT: 5, SL: 10, mean: 5.676732681935291, std: 0.559035872061135, SR: 10.154505221651492
Forecast: 10,  H-L: 25
PT: 5, SL: 10, mean: 5.5977174905157145, std: 0.6740844437673235, SR: 8.304178418999241
Forecast: 10,  H-L: 50
PT: 5, SL: 10, mean: 4.958745210783436, std: 2.6331193400314232, SR: 1.8832208382641173
Forecast: 10,  H-L: 100
PT: 5, SL: 10, mean: 3.329924735884716, std: 4.938865864991082, SR: 0.6742286239212792
Forecast: 5,  H-L: 5
PT: 5, SL: 10, mean: 5.597447060527242, std: 0.5063125822631507, SR: 11.055318901038147
Forecast: 5,  H-L: 10
PT: 5, SL: 10, mean: 5.570840055316623, std: 0.5635647627426492, SR: 9.885004215319503
Forecast: 5,  H-L: 25
PT: 5, SL: 10, mean: 5.045121562774392, std: 2.1533787584397976, SR: 2.342886286493217
Forecast: 5,  H-L: 50
PT: 5, SL: 10, mean: 3.626904010993998, std: 4.444554452638991, SR: 0.8160332041472669
Forecast: 5

[(5, 10, -3.111735025113942, 7.2874530657989345, -0.4269989798929563)]

### 13.1
a) Using the code snippets, we can set the profit-taking to 5 (since we want to close position if price rises by 5) and stop-loss to 10 (close position when prices drops by 10). The batch() function then calculates the Sharpe Ratios for various parameters. We will want to take the opposite sign of the Sharpe Ratios since we have a short position and will profit from the stop-loss and face a loss from the profit-take.  
These thresholds are reasonable if the forecasted price has a negative long-run equilibrium. In addition, the smaller half-life, the higher the Sharpe Ratio.  
  
b) Geometric Brownian motion is an alternative process that might make sense for these thresholds.  
TODO: is this correct?

### 13.2 - 13.4
TODO: Figure out how to fit time series to an O-U process.  
TODO: Find an additional dataset to compare with